# Подготовка данных

## Считаем данные в исходном csv формате

In [ ]:
!pip install findspark

Перезапустить kernel после установки!

In [ ]:
import findspark
findspark.init()

In [ ]:
!hdfs dfs -mkdir -p /user/ubuntu/data

In [ ]:
!hadoop distcp s3a://otus-bucket-b1gcq77orp7o97v8lnkm/ /user/ubuntu/data

In [ ]:
!hdfs dfs -ls /user/ubuntu/data

In [ ]:
!hadoop distcp s3a://otus-bucket-b1gcq77orp7o97v8lnkm/questions.csv /user/ubuntu/data/questions.csv

In [ ]:
!hdfs dfs -ls

In [ ]:
!hdfs dfs -ls data

Создадим SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS")
        .getOrCreate()
)

Загрузим данные

In [ ]:
df = spark.read.csv("data/train_1000k.csv", inferSchema=True, header=True)

Поделим данные на 10 разделов и сохраним в формате parquet

In [ ]:
%%time
df.count()

In [ ]:
(
    df
        .repartition(10)
        .write
        .mode("overwrite")
        .parquet("data/train.parquet")
)

Проверим размер сохраненного файла

In [ ]:
!hdfs dfs -ls -h data/train.parquet

In [ ]:
df_from_parquet = spark.read.parquet("data/train.parquet")

In [ ]:
%%time
df_from_parquet.count()

In [ ]:
df_from_parquet.show()